### Import libraries

In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Load data

In [3]:
dataset = pd.read_csv('IMDB Dataset.csv')
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### Preprocess data

In [4]:
dataset['sentiment'] = dataset['sentiment'].map({'positive': 1, 'negative': 0})

In [5]:
reviews = dataset['review'].values
labels = dataset['sentiment'].values

### Split data

In [6]:
X_train_texts = reviews[:40000]
y_train = labels[:40000]
X_test_texts = reviews[40000:]
y_test = labels[40000:]

### Tokenize

In [7]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X_train_texts)

In [8]:
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1 

In [9]:
X_train_sequences = tokenizer.texts_to_sequences(X_train_texts)
X_train_padded = pad_sequences(X_train_sequences, padding='post', maxlen=2500)

X_test_sequences = tokenizer.texts_to_sequences(X_test_texts)
X_test_padded = pad_sequences(X_test_sequences, padding='post', maxlen=2500)

### Model

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=10, input_length=2500),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
model.fit(X_train_padded, y_train, epochs=20, validation_data=(X_test_padded, y_test), verbose=1)

Epoch 1/20
1250/1250 [==============================] - 30s 23ms/step - loss: 0.6520 - accuracy: 0.6418 - val_loss: 0.5338 - val_accuracy: 0.7375
Epoch 2/20
1250/1250 [==============================] - 27s 21ms/step - loss: 0.3966 - accuracy: 0.8514 - val_loss: 0.3778 - val_accuracy: 0.8265
Epoch 3/20
1250/1250 [==============================] - 23s 18ms/step - loss: 0.2832 - accuracy: 0.8947 - val_loss: 0.2869 - val_accuracy: 0.8925
Epoch 4/20
1250/1250 [==============================] - 22s 18ms/step - loss: 0.2360 - accuracy: 0.9121 - val_loss: 0.2673 - val_accuracy: 0.8987
Epoch 5/20
1250/1250 [==============================] - 25s 20ms/step - loss: 0.2067 - accuracy: 0.9236 - val_loss: 0.2570 - val_accuracy: 0.9037
Epoch 6/20
1250/1250 [==============================] - 32s 26ms/step - loss: 0.1830 - accuracy: 0.9337 - val_loss: 0.2680 - val_accuracy: 0.8959
Epoch 7/20
1250/1250 [==============================] - 26s 21ms/step - loss: 0.1638 - accuracy: 0.9401 - val_loss: 0.2666 -

In [15]:
model = tf.keras.models.load_model('model.h5')

### Prediction

In [47]:
review = [
    "Scene with the CGI Leia was horrible. I wouldn't recommend this movie it to others.",
    "The movie was great. I enjoyed every secon of it although actors play wasn't perfect but plot made up for it. I would recommend it to others.",
    "I loved this movie. It was so funny and entertaining.",
    "This movie was terrible. The plot was boring and the acting was awful.",
    "Funny movie but not an amazing one. Best to watch with your friends.",
]

review_sequence = tokenizer.texts_to_sequences(review)
review_padded = pad_sequences(review_sequence, maxlen=2500, padding='post')

prediction = model.predict(review_padded)
print(prediction.flatten())

1/1 [==============================] - 0s 15ms/step
[0.07068006 0.90992635 0.7950106  0.00118283 0.5537654 ]
